# View Structure Subsequences 

#### Set file name of the dataset, and run this cell once to initialize the program.
- Dataset columns should have "ProteinID" and "PeptideSequence"

In [ ]:
from scripts.sequence_viewer import *

filename = 'Peptides sequence_human urine solution digestion.csv' 
data = pd.read_csv(os.path.join("data",filename)) 

#### Choose any ProteinID to view selected sequences
- default color order is: purple | yellow | blue | green | (repeated if there are more than 4 columns of intensities)
- color is determined by the maximum value amoung those columns
- if no numbers exist for the subsequence (only overnight digestion), it is colored cyan 
- if no "min" columns exist, then all sequences will be highlighted in the first listed color

#### Set custom colors by modifying the colors list
- the last color is used when no value is present for any time
- use hexidecimal color codes: https://www.color-hex.com/

In [ ]:
proteinID = 'P02768'
colors = ['#A020F0', 'yellow', 'blue', 'green', 'cyan']

getPepView(proteinID, data, colors=colors, table=True) #set table=False to hide the table preview

# Get GRAVY Differences

- By default, the program will assume that Table1, Table2, and the FASTA files are in the "data" folder
    - these locations can be customized to specify other paths
- Output will be saved in the same folder location as Table 1
- Alternatively, the same results can be obtained by running gravy_diff.py in the scripts folder
- Table1 and Table2 should have columns named "ProteinID" and "PeptideSequence"

In [ ]:
from scripts.gravy_diff import *

#set file names
table1 = "Table 1_Peptides sequence_human urine solution digestion .csv"
table2 = "Table 2_20211204_HUVariousDigestionTime_pr_matrix.csv"
fasta = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta"

#replace "data" with a custom path if the files are located in another folder (in quotations)
table1Path = os.path.join("data",table1)
table2Path = os.path.join("data", table2)
fastaPath = os.path.join("data", fasta)

table1diff = getGRAVYdiffs(fastaPath, table1Path, table2Path)

#append results to table2:
data2 = pd.read_csv(table2Path)
if "ProteinID" not in data2.columns: data2 = data2.rename(columns={"PG.UniProtIds": "ProteinID", "Stripped.Sequence": "PeptideSequence"})
#handling isomers:
table1diff["UPID"] = [i.split(';')[0] for i in table1diff["ProteinID"].values]
data2["UPID"] = [i.split(';')[0] for i in data2["ProteinID"].values]

subset = table1diff.drop_duplicates("UPID")[["UPID", "ProteinSequence", "SequenceGRAVY", "GRAVYdifference", "GRAVYdifference2"]]
table2diff = pd.merge(data2,subset, on=["UPID"], how='left')
table2diff.drop('UPID', axis=1).to_csv(table2Path[:-4]+'_GRAVY.csv', index=False)

# Get Peptide Sequence Percentage

In [ ]:
from scripts.gravy_diff import *

#set file names:
filename = "labled proteins data set.csv" 
fasta = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta"

#replace "data" with a custom path if the files are located in another folder
filepath = os.path.join("data",filename) 
fastaPath = os.path.join("data", fasta)

data = pd.read_csv(filepath)
if "ProteinSequence" not in data.columns or "PeptideSequence" not in data.columns:
    data = process(data, fastaPath)
data = peptidePercentage(data)
data.to_csv(os.path.join("output", filename[:-4])+"_Percentage.csv", index=False)

# Get PeptideSequence Distances

#### Set file and Fasta name and run this cell to get peptide distances from their overall center of mass
- Initial dataset should include a "ProteinID" and "PeptideSequence" column
- When completed, the results will be saved as csv files in the output folder
- The output only keeps ProteinIDs that have results in Protein Data Bank
- This process takes a long time to complete (some hours)

In [ ]:
from protds.peptide_distances import *
from scripts.gravy_diff import process, pepPositions, peptidePercentage

filename = "labled proteins data set.csv" #set file name
fastaname = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta" #set fasta name

#if the files are not in the "data" folder, replace "data" with the full path to that location
fastapath = os.path.join("data", fastaname)
filepath = os.path.join("data", filename)

df = pepPositions(process(pd.read_csv(filepath), fastapath))
times = ['1st 15min','2nd 15min', '3rd 15min', '4th 15min', 'OvernightDigestion']#change to match the data's column names
try:
    getCenterDistByTime(filename, df, times)
except TypeError:
    invalid = [i[0] for i in proteins.items() if i[1].structures==None]
    for i in invalid: del proteins[i]
    getCenterDistByTime(filename, df, times)

# SVD Plane
Find a plane that minimizes the total perpendicular distance from the points to the plane
- A total least squares problem solved with Singular Value Decompostion
- Run this cell to get the distances from each sequence to their plane

In [ ]:
from protds.peptide_distances import *
from scripts.gravy_diff import process, pepPositions, filterSequences

#set names for the data and fasta files
filename = "Table 2_20211204_HUVariousDigestionTime_pr_matrix.csv"
fastaname = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta"

#replace "data" with a custom path if the files are located in another folder (in quotations)
filepath = os.path.join("data", filename)
fastapath = os.path.join("data", fastaname)

#results:
data2 = process(pd.read_csv(filepath), fastapath, "PG.UniProtIds", "Stripped.Sequence")
filtered = filterSequences(data2)
table2 = pepPositions(filtered[0])
try:
    planeDists = pepPlaneDist(table2)
except TypeError:
    invalid = [i[0] for i in proteins.items() if i[1].structures==None]
    for i in invalid: del proteins[i]
    planeDists = pepPlaneDist(table2, True)

planeDists.drop("PepMid", axis=1).to_csv(os.path.join("output", filename[:-4]+'_Plane.csv'))
filtered[1].to_csv(os.path.join("filtered", filename[:-4]+'_Plane_Excluded.csv'))

# SVD Plane Visualization

In [ ]:
#run this cell once for initializations 
from protds.peptide_distances import *
from scripts.gravy_diff import process, pepPositions, filterSequences
from ipywidgets import *
#%matplotlib notebook #for JupyterNotebook

#set names for the data and fasta files
filename = "Table 2_20211204_HUVariousDigestionTime_pr_matrix.csv"
fastaname = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta"

#replace "data" with a custom path if the files are located in another folder (in quotations)
filepath = os.path.join("data", filename)
fastapath = os.path.join("data", fastaname)

#results:
data2 = process(pd.read_csv(filepath), fastapath, "PG.UniProtIds", "Stripped.Sequence")
filtered = filterSequences(data2)
df = pepPositions(filtered[0])

In [ ]:
protid = "O00391" #select a ProteinID of interest

%matplotlib widget 
protgroup = (protid, df[[protid in i for i in df.ProteinID]])
if searchPDB(protid, False, protgroup[1].ProteinSequence.values[0]) != False:
    plotPlane(protgroup)

# Intensity Summaries
Assumed file structures:
- There are multiple data files corresponding to incubation times, each with the same naming convention, but starting with different times. Ex:
    - "5min incubation_Plane_GRAVY_Percentage.csv"
    - "15min incubation_Plane_GRAVY_Percentage.csv"
    - "24h incubation_Plane_GRAVY_Percentage.csv"
      
      
- "Average intensity" is stored in one Excel file (intensityFile) with multiple sheets. Each sheet is similarly named, but starting with different times. Ex:
    - "5min incubation"
    - "15min incubation"
    - "24h incubation"
      
       
- Please make sure the times are named consistantly between the sheets and files (set in the times list)  
  
  
- filename and intensitySheetName is the name following the times, including any spaces  
  
Results (time incubation_summary.csv) will be saved in the "output" folder. 

In [ ]:
from scripts.gravy_diff import intensitySummary
import pandas as pd
import os

path = "data" #set location of the data files
#set file names:
classificationFile = "Protein classification.xlsx"
intensityFile = "Protein intensity.xlsx" 

times = ["5min", "15min", "30min", "24h"] #set to match begining of file names
intensitySheetName = " incubation" #name of sheets for intensityFile
filename = " incubation_Plane_GRAVY_Percentage.csv"  #name of data to be summarized

#Processing:
classes = pd.read_excel(os.path.join(path, classificationFile), sheet_name = None)
intensities = pd.read_excel(os.path.join(path, intensityFile), sheet_name = None)
allclasses = pd.concat([df for df in classes.values()], ignore_index=True)
classdict = dict(zip(allclasses['PG.UniProtIds'], allclasses.kmean_kernal))

for i in times:
    data = pd.read_csv(os.path.join("data", i+filename))
    intensitySummary(classdict, intensities[i+intensitySheetName], data).to_csv(os.path.join("output", i+" incubation_summary.csv"), index=False)

### After the summaries are saved to the output folder, these following cells can be ran for correlation and trend 

In [ ]:
# Correlations:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

times = ["5min", "15min", "30min", "24h"]
filename = " incubation_summary.csv"

cols = ['SequenceGRAVY', 'GRAVYdifference','PeptidePercentage', 'Average Intensity','Average DistanceToPlane']

for i in times:
    file = i+filename
    data = pd.read_csv(os.path.join("output", file))
    
    print(f"TIME = {i}",'\n', "correlation:")
    #correlation and heatmaps
    display(data[cols].corr())
    sns.heatmap(data[cols].corr(), cmap='coolwarm', vmax=1.0, vmin=-1.0 )
    plt.show()

In [ ]:
# Trends: line correlations are saved in trend_dist_intensity.csv in the output folder
# (After running this cell once, line plots can be generated in the following cell)
import matplotlib.pyplot as plt
import pandas as pd
import os
from functools import reduce 

times = ["5min", "15min", "30min", "24h"]
filename = " incubation_summary.csv"
files = [pd.read_csv(os.path.join("output", i+filename))[['ProteinID', 'Average Intensity','Average DistanceToPlane']] for i in times]

def relabelIDs (ids1, ids2): 
    labs = []
    for i in ids2:
        add = True
        for j in ids1:
            if i in j.split(";"): 
                labs+= [j]
                add = False
                break
        if add: labs +=[i]
    return labs

for i in range(1,len(files)):
    files[i].ProteinID = relabelIDs(files[i-1].ProteinID, files[i].ProteinID)

distances = [df[["ProteinID", "Average DistanceToPlane"]] for df in files]
intensities = [df[["ProteinID", "Average Intensity"]] for df in files]
dists = reduce(lambda  left,right: pd.merge(left,right,on=['ProteinID'], how='outer'), distances)
intens = reduce(lambda  left,right: pd.merge(left,right,on=['ProteinID'], how='outer'), intensities)
dists.columns = ["ProteinID"]+times
intens.columns = ["ProteinID"]+times

#correlations
df = pd.concat([dists, intens[intens.columns[1:]]], keys=['Avg DistanceToPlane','Avg Intensity'], axis=1)
df["Correlation"] = [round(pd.DataFrame([p[1:5],p[5:9]]).T.corr()[0][1], 5) for p in df.values]
df.loc[~df['Correlation'].between(-1, 1, inclusive="both"), "Correlation"] = None
#classes
pat = r'({})'.format('|'.join(classdict.keys()))
extracted = df[df.columns[0]].str.extract(pat, expand=False).dropna()
df['Classification'] = extracted.apply(lambda x: classdict[x]).reindex(df.index)

df.to_csv(os.path.join("output", "trend_dist_intensity.csv"), index=False)
df.columns = list(df.columns.droplevel())[:-2]+["Correlation", "Classification"]

In [ ]:
# Line Plots
id = "O00391" #select a ProteinID

selection = df[df["ProteinID"]==id]
print("Avg Distance:")
selection.iloc[:, 1:5].T.plot.line(legend=False)
plt.show()

print("Avg Intensity:")
selection.iloc[:, 5:9].T.plot.line(legend=False)
plt.show()

# Plane 1:
"Use the sequence in table 2 to define plane 1, calculate the distance of each sequenced peptide to this plane. (The sequence with the highest intensity at 1st 15min should on this plane.) ... the distance of the other sequences to this plane should be minimum.  "

In [ ]:
from protds.peptide_distances import *
from scripts.gravy_diff import process, pepPositions, filterSequences

#set names for the data and fasta files
filename = "Table 2_20211204_HUVariousDigestionTime_pr_matrix.csv"
fastaname = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta"

#replace "data" with a custom path if the files are located in another folder (in quotations)
filepath = os.path.join("data", filename)
fastapath = os.path.join("data", fastaname)

#results:
data2 = process(pd.read_csv(filepath), fastapath, "PG.UniProtIds", "Stripped.Sequence")
filtered = filterSequences(data2)
table2 = pepPositions(filtered[0])
try:
    planeDists = pepPlaneDist(table2, True).rename(columns={"DistanceToPlane": "DistanceToPlane1"})
except TypeError:
    invalid = [i[0] for i in proteins.items() if i[1].structures==None]
    for i in invalid: del proteins[i]
    planeDists = pepPlaneDist(table2, True).rename(columns={"DistanceToPlane": "DistanceToPlane1"})

planeDists.drop("PepMid", axis=1).to_csv(os.path.join("output", filename[:-4]+'_Plane1.csv'))
filtered[1].to_csv(os.path.join("filtered", filename[:-4]+'_Plane1_Excluded.csv'))

### Angle between Plane1 and Plane2

In [ ]:
table1 = "Table 1_Peptides sequence_human urine solution digestion .csv" #used for plane2
table1Path = os.path.join("data", table1)
    
#table1-table2 sequences:
data1 = pepPositions(filterSequences(process(pd.read_csv(table1Path), fastapath))[0])
data1["UPID"] = [i.split(';')[0] for i in data1["ProteinID"].values]
data2["UPID"] = [i.split(';')[0] for i in data2["ProteinID"].values]
data1Minus2 = pd.merge(data1, data2, on=["UPID", "PeptideSequence"], how="outer", suffixes=('','_2'), indicator=True).query('_merge=="left_only"')[data1.columns]

try:
    angles = getAngles(planeDists, data1Minus2)
except TypeError:
    invalid = [i[0] for i in proteins.items() if i[1].structures==None]
    for i in invalid: del proteins[i]
    angles = getAngles(planeDists, data1Minus2)
    
angles.drop("PepMid", axis=1).to_csv(os.path.join("output", filename[:-4]+'_Plane1.csv'))

# Plane 2: 
"For each protein in table 1 (human urine solution digestion), after removing the peptides sequences found in table 2, use the left sequences to define plane 2, calculate the distance of each sequenced peptide to this plane. "

In [ ]:
from protds.peptide_distances import *
from scripts.gravy_diff import process, pepPositions, filterSequences

#set file names
table1 = "Table 1_Peptides sequence_human urine solution digestion .csv"
table2 = "Table 2_20211204_HUVariousDigestionTime_pr_matrix.csv"
fasta = "uniprot-human+taxonomy__Homo+sapiens+(Human)+[9606]_-filtered-revi--.fasta"

#replace "data" with a custom path if the files are located in another folder (in quotations)
table1Path = os.path.join("data",table1)
table2Path = os.path.join("data", table2)
fastaPath = os.path.join("data", fasta)

#table1-table2 sequences:
data1 = pepPositions(process(pd.read_csv(table1Path), fastaPath))
data2 = pd.read_csv(table2Path) 
if "ProteinID" not in data2.columns: data2 = data2.rename(columns={"PG.UniProtIds": "ProteinID", "Stripped.Sequence": "PeptideSequence"})
data1["UPID"] = [i.split(';')[0] for i in data1["ProteinID"].values]
data2["UPID"] = [i.split(';')[0] for i in data2["ProteinID"].values]

df = pd.merge(data1, data2, on=["UPID", "PeptideSequence"], how="outer", suffixes=('','_2'), indicator=True).query('_merge=="left_only"')[data1.columns]
filtered = filterSequences(df)
try:
    diffdist = pepPlaneDist(filtered[0], False).drop(["PepMid", "UPID"], axis=1).rename(columns={"DistanceToPlane": "DistanceToPlane2"})
except TypeError:
    invalid = [i[0] for i in proteins.items() if i[1].structures==None]
    for i in invalid: del proteins[i]
    diffdist = pepPlaneDist(filtered[0], False).drop(["PepMid", "UPID"], axis=1).rename(columns={"DistanceToPlane": "DistanceToPlane2"})

diffdist.to_csv(os.path.join("output", table1[:-4]+'_Plane2.csv'))
filtered[1].to_csv(os.path.join("filtered", table1[:-4]+'_Plane2_Excluded.csv'))